In [ ]:
import sys
# print(sys.path)
sys.path.append("/home/abdullahm/jaleel/Review_analysis")

['/home/abdullahm/jaleel/Review_analysis/notebooks', '', '/home/abdullahm/miniconda3/envs/aaai2/lib/python3.11/site-packages', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python313.zip', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python3.13', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python3.13/lib-dynload', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python3.13/site-packages']


In [2]:
from src.util.config import load_config

cfg = load_config()


### convert Raw data to json and save it


In [5]:
import pickle as pkl
import json

def pkl_to_json(src_path, target_path):
    # load pickle (binary mode)
    with open(src_path, 'rb') as f:  
        data = pkl.load(f)
    
    # save JSON (text mode)
    with open(target_path, 'w') as f:
        json.dump(data, f, indent=2, default=str)  
        # default=str converts unsupported types to string


In [8]:
pkl_to_json(f"{cfg["paths"]['data_dir']}/raw/test_all.pkl", f"{cfg["paths"]["data_dir"]}/test.json")

### split question and answer in dataset


In [ ]:
import json
import re

# Paths to your JSON files
train_json_path = cfg["paths"]['data_dir']+"/train.json"
test_json_path  = cfg["paths"]['data_dir']+"/test.json"

output_train_json = cfg["paths"]['data_dir']+"/train_split.json"
output_test_json  = cfg["paths"]['data_dir']+"/test_split.json"

# List of questions
questions = [
    "According to you , what is going well in this course",
    "Is there anything you wish to be changed",
    "Share your suggestions on how the experience could have improved",
    "What helped you learn in this course",
    "What changes would improve your learning",
    "Comments on labs (if any) with respect to the course and your learning",
    "What suggestions do you have for another student considering taking this course",
    "Comments on teaching (if any) assistant(s) with respect to the course and your learning",
    "Comments on tutorials (if any) with respect to the course and your learning",
    "Any comment you want to make on TAs work and labs",
    "According to you, what is going well in the course",
    "According to you what is going well in the course",
    "What changes would improve learning",
    "What helped you learned in this course",
    "According to you, what is going well in this course? Name atleast one."
]

def split_sentence_to_qa(data):
    new_data = []
    for item in data:
        sentence = item.get("sentence", "")
        matched_question = None
        for q in questions:
            # Case-insensitive match; remove extra spaces
            if sentence.lower().startswith(q.lower()):
                matched_question = q
                break
        if matched_question:
            # Extract answer
            answer = sentence[len(matched_question):].strip(" :,.-")
            item["question"] = matched_question
            item["answer"] = answer
        else:
            # If no match, leave question empty, answer = sentence
            print("NO MATCH", item.get("sentence") )
            # item["question"] = ""
            # item["answer"] = sentence
        new_data.append(item)
    return new_data

def process_json(input_path, output_path):
    # Load JSON
    with open(input_path, "r") as f:
        data = json.load(f)
    
    # Split sentences
    new_data = split_sentence_to_qa(data)
    
    # Save new JSON
    with open(output_path, "w") as f:
        json.dump(new_data, f, indent=4)
    
    print(f"Saved {output_path}")

# Process train and test JSON
process_json(train_json_path, output_train_json)
process_json(test_json_path, output_test_json)


NO MATCH According to you, what is going well in this course? Name atleast one. I'm learning great stuff! Theory courses in IIIT Delhi are really great in my opinion, and the lesser attendance is like icing on the cake (I prefer classes with fewer students). Homeworks, lectures etc. are all going pretty well.
NO MATCH According to you, what is going well in this course? Name atleast one. I'm learning great stuff! Theory courses in IIIT Delhi are really great in my opinion, and the lesser attendance is like icing on the cake (I prefer classes with fewer students). Homeworks, lectures etc. are all going pretty well.
NO MATCH According to you, what is going well in this course? Name atleast one. I'm learning great stuff! Theory courses in IIIT Delhi are really great in my opinion, and the lesser attendance is like icing on the cake (I prefer classes with fewer students). Homeworks, lectures etc. are all going pretty well.
NO MATCH According to you, what is going well in this course? Name 